In [1]:
import os
import json
import glob
import torch
import cv2
import numpy as np
from pathlib import Path
from tqdm import tqdm
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
import ujson
import ray
import math
from dataclasses import dataclass
from typing import Union

# Import your CellViT modules
from cellvit.models.cell_segmentation.cellvit_virchow import CellViTVirchow
from cellvit.utils.tools import unflatten_dict
from cellvit.inference.inference_disk import CellViTInference
from cellvit.inference.postprocessing_cupy import (
    BatchPoolingActor,
    DetectionCellPostProcessorCupy,
)
from cellvit.config.config import TYPE_NUCLEI_DICT_PANNUKE

class region_dataset(Dataset):
    def __init__(self, image_path, transform=None, patch_size=256):
        img_bgr = cv2.imread(image_path)
        img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
        self.img_np = np.array(img_rgb)
        self.patch_size = patch_size
        self.region_coords = []
        self.transform = transform
        h, w, _ = self.img_np.shape
        
        # Calculate number of patches needed
        for i in range(0, h, self.patch_size):
            for j in range(0, w, self.patch_size):
                self.region_coords.append((i, j))

    def __len__(self):
        return len(self.region_coords)
    
    def __getitem__(self, idx):
        x, y = self.region_coords[idx]
        patch = self.img_np[x:min(x + self.patch_size, self.img_np.shape[0]),
                            y:min(y + self.patch_size, self.img_np.shape[1])]
        
        # Pad if necessary
        if patch.shape[0] < self.patch_size or patch.shape[1] < self.patch_size:
            patch = np.pad(patch, 
                          ((0, self.patch_size - patch.shape[0]), 
                           (0, self.patch_size - patch.shape[1]), 
                           (0, 0)), 
                          mode='constant')
        
        if self.transform:
            patch = self.transform(patch)
        
        metadata = {
            "col": y // self.patch_size,
            "row": x // self.patch_size,
            "position_x": x,
            "position_y": y,
            "patch_size": self.patch_size
        }
    
        return patch, metadata

@dataclass
class WSIMetadata:
    slide_path: Union[str, Path]
    metadata: dict

def get_wsi_metadata(image_path, patch_size=256, downsample=1, patch_overlap=0, label_map=None, normalize_stains=False):
    # Get image dimensions
    img = cv2.imread(image_path)
    h, w = img.shape[:2]
    
    n_cols = math.ceil(w / patch_size)
    n_rows = math.ceil(h / patch_size)
    
    slide_mag = 40  
    slide_mpp = 0.25  
    resulting_mpp = slide_mpp * downsample

    wsi_metadata = {
        "orig_n_tiles_cols": n_cols,
        "orig_n_tiles_rows": n_rows,
        "base_magnification": slide_mag,
        "downsampling": downsample,
        "label_map": label_map if label_map is not None else {},
        "patch_overlap": patch_overlap * 2,
        "patch_size": patch_size,
        "base_mpp": slide_mpp,
        "target_patch_mpp": resulting_mpp,
        "stain_normalization": normalize_stains,
        "magnification": slide_mag / (downsample * 1.0),
        "level": 8,
    }

    return wsi_metadata

def default_collate_fn(batch):
    patches = []
    metadatas = []
    for patch, metadata in batch:
        patches.append(patch)
        metadatas.append(metadata)
    return torch.stack(patches), metadatas

def process_single_region(image_path, model, device, run_conf, infer_cell, batch_size=8):  # Reduced further
    """Process a single region image and save GeoJSON results"""
    
    image_path_obj = Path(image_path)
    output_dir = image_path_obj.parent
    region_name = image_path_obj.stem
    
    geojson_path = output_dir / f"{region_name}_cells.geojson"
    # Uncomment to skip existing files
    # if geojson_path.exists():
    #     print(f"Skipping {region_name}, GeoJSON already exists")
    #     return
    
    print(f"Processing region: {region_name}")
    
    # Setup transforms and dataset
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225]
        )
    ])

    dataset = region_dataset(image_path=image_path, transform=transform, patch_size=256)
    dataloader = DataLoader(dataset=dataset, batch_size=batch_size, num_workers=0, 
                           collate_fn=default_collate_fn, pin_memory=True)

    wsi_metadata = get_wsi_metadata(
        image_path=image_path,
        patch_size=256,
        downsample=1,
        patch_overlap=0,
        label_map={"tumor": 1, "normal": 0},
        normalize_stains=False,
    )
    wsi = WSIMetadata(
        slide_path=image_path,
        metadata=wsi_metadata,
    )

    postprocessor = DetectionCellPostProcessorCupy(
        wsi=wsi,
        nr_types=run_conf["data"]["num_nuclei_classes"],
        resolution=0.25,
        classifier=None, 
        binary=False,
    )

    # Use fewer ray actors to avoid resource contention
    ray_actors = min(2, batch_size)  # Reduced from 4-8 to max 2
    
    try:
        # Initialize ray once with appropriate resources
        if not ray.is_initialized():
            ray.init(num_cpus=ray_actors, num_gpus=0.1, ignore_reinit_error=True)
        
        batch_pooling_actors = [
            BatchPoolingActor.remote(postprocessor, run_conf)
            for _ in range(ray_actors)
        ]
        
        call_ids = []
        with torch.inference_mode():
            for batch_num, (patches, metadata) in enumerate(tqdm(dataloader, desc=f"Processing {region_name}")):
                batch_actor = batch_pooling_actors[batch_num % ray_actors]
                patches = patches.to(device, non_blocking=True)
                
                predictions = model(patches, retrieve_tokens=True)
                predictions = infer_cell.apply_softmax_reorder(predictions=predictions)
                predictions = {k: v.cpu() if isinstance(v, torch.Tensor) else v 
                              for k, v in predictions.items()}
                
                call_id = batch_actor.convert_batch_to_graph_nodes.remote(predictions, metadata)
                call_ids.append(call_id)

            # Process results in smaller chunks to avoid memory issues
            inference_results = []
            for i in range(0, len(call_ids), 4):  # Process 4 at a time
                chunk = call_ids[i:i+4]
                inference_results.extend(ray.get(chunk))

        # Result processing
        cell_dict_wsi = []
        cell_dict_detection = []
        
        label_map = TYPE_NUCLEI_DICT_PANNUKE 
        label_map = {int(k): v for k, v in label_map.items()}
        
        graph_data = {
            "cell_tokens": [],
            "positions": [],
            "metadata": {
                "wsi_metadata": wsi.metadata,
                "nuclei_types": label_map,
            },
        }
        
        for batch_results in inference_results:
            (
                batch_complete_dict,
                batch_detection,
                batch_cell_tokens,
                batch_cell_positions,
            ) = batch_results
            
            cell_dict_wsi.extend(batch_complete_dict)
            cell_dict_detection.extend(batch_detection)
            graph_data["cell_tokens"].extend(batch_cell_tokens)
            graph_data["positions"].extend(batch_cell_positions)

        keep_idx = infer_cell._post_process_edge_cells(cell_list=cell_dict_wsi)
        cell_dict_wsi = [cell_dict_wsi[idx_c] for idx_c in keep_idx]
        cell_dict_detection = [cell_dict_detection[idx_c] for idx_c in keep_idx]
        graph_data["cell_tokens"] = [graph_data["cell_tokens"][idx_c] for idx_c in keep_idx]
        graph_data["positions"] = [graph_data["positions"][idx_c] for idx_c in keep_idx]
        
        final_cell_dict = {
            "wsi_metadata": wsi.metadata,
            "type_map": label_map,
            "cells": cell_dict_wsi,  
        }
        
        # Save results
        geojson_list = infer_cell._convert_json_geojson(final_cell_dict["cells"], True)
        
        with open(geojson_path, "w") as outfile:
            ujson.dump(geojson_list, outfile)
        
    except Exception as e:
        print(f"Error processing {region_name}: {str(e)}")
        # Create empty file on error to avoid reprocessing
        with open(geojson_path, "w") as outfile:
            ujson.dump([], outfile)
    finally:
        # Don't shutdown ray here - let the main function handle it
        torch.cuda.empty_cache()

def main():
    # Load model
    model_path = "/home/shivam/nuc_seg/CellViT-plus-plus-main/checkpoints/CellViT-Virchow-x40-AMP-001.pth"
    model_checkpoint = torch.load(model_path, map_location="cpu")
    run_conf = unflatten_dict(model_checkpoint["config"], ".")
    
    model = CellViTVirchow(
        model_virchow_path=None,
        num_nuclei_classes=run_conf["data"]["num_nuclei_classes"],
        num_tissue_classes=run_conf["data"]["num_tissue_classes"]
    )
    model.load_state_dict(model_checkpoint["model_state_dict"])
    
    device = torch.device('cuda:2' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)
    model.eval()
    
    run_conf["model"]["token_patch_size"] = model.patch_size
    
    infer_cell = CellViTInference(model_path=model_path, gpu=2)
    
    # Find all region folders
    base_directory = "/home/shivam/nuc_seg/CellViT-plus-plus-main/inference_data"
    region_folders = [f for f in os.listdir(base_directory) 
                  if os.path.isdir(os.path.join(base_directory,f)) and f.endswith("_10_regions")]
    
    print(f"Found {len(region_folders)} region folders")
    
    try:
        # Process all region images
        for folder in tqdm(region_folders, desc="Processing folders"):
            folder_path = os.path.join(base_directory, folder)
            image_files = glob.glob(os.path.join(folder_path, "*.png"))
            
            print(f"Processing {len(image_files)} images in {folder}")
            
            for image_path in tqdm(image_files, desc=f"Processing {folder}"):
                process_single_region(
                    image_path=image_path,
                    model=model,
                    device=device,
                    run_conf=run_conf,
                    infer_cell=infer_cell,
                    batch_size=16  # Further reduced batch size
                )
    
    finally:
        # Clean up ray at the end
        if ray.is_initialized():
            ray.shutdown()
        torch.cuda.empty_cache()
    
    print("All regions processed successfully!")

if __name__ == "__main__":
    main()

No checkpoint provided!
2025-09-08 14:35:16,601 [INFO] - Loading model: /home/shivam/nuc_seg/CellViT-plus-plus-main/checkpoints/CellViT-Virchow-x40-AMP-001.pth
No checkpoint provided!
2025-09-08 14:35:22,333 [INFO] - <All keys matched successfully>
2025-09-08 14:35:22,849 [INFO] - Based on the hardware we limit the batch size to a maximum of:
2025-09-08 14:35:22,851 [INFO] - 8
2025-09-08 14:35:22,852 [INFO] - Loading inference transformations


2025-09-08 14:35:28,033	INFO worker.py:1724 -- Started a local Ray instance.


2025-09-08 14:35:32,303 [INFO] - Using 4 ray-workers
Found 15 region folders


Processing folders:   0%|                                | 0/15 [00:00<?, ?it/s]

Processing 10 images in TCGA-25-1312-01Z-00-DX1.733EC7A7-0FC8-4DDC-B366-DF5A45D6BB4E_10_regions


...

Processing region: TCGA-ER-A2NF-01Z-00-DX2.01ACC455-4161-4EF1-8478-3C4DF709F6D6_x73071_y27401




Processing TCGA-ER-A2NF-01Z-00-DX2.01ACC455-4161-4EF1-8478-3C4DF709F6D6_x73071_y

Processing TCGA-ER-A2NF-01Z-00-DX2.01ACC455-4161-4EF1-8478-3C4DF709F6D6_x73071_y


2025-09-08 14:51:55,096 [INFO] - Initializing Cell-Postprocessor
2025-09-08 14:51:55,103 [INFO] - Finding edge-cells for merging
2025-09-08 14:51:55,120 [INFO] - Removal of cells detected multiple times
2025-09-08 14:51:55,136 [INFO] - Iteration 0: Found overlap of # cells: 0
2025-09-08 14:51:55,137 [INFO] - Found all overlapping cells



Processing TCGA-ER-A2NF-01Z-00-DX2.01ACC455-4161-4EF1-8478-3C4DF709F6D6_10_regio
Processing folders: 100%|███████████████████████| 15/15 [16:22<00:00, 65.52s/it]


All regions processed successfully!
